### 1. 각 군집의 centroid 부터 거리 계산 (주성분 활용)

In [122]:
import pandas as pd
import numpy as np

pca_result = pd.read_csv('./data/minmax_scaling_PCA_label.csv', index_col=0)

# kmeans centroid numpy array 불러오기
centroids = np.load('./data/kmeans_centroid.npy')

In [123]:
# 클러스터 레이블 별로 데이터프레임 나누기
pca_cluster_0 = pca_result[pca_result['cluster'] == 0]
pca_cluster_1 = pca_result[pca_result['cluster'] == 1]
pca_cluster_2 = pca_result[pca_result['cluster'] == 2]
pca_cluster_3 = pca_result[pca_result['cluster'] == 3]

In [124]:
# 거리 계산 함수 생성
def calculate_distance(a, b):
    distance = 0
    for i in range(len(a)):
        distance += (a[i] - b[i]) ** 2
    return distance ** 0.5

In [101]:
# 오류 메시지 비활성화
import pandas as pd
pd.options.mode.chained_assignment = None

In [125]:
# 각 cluster centroid로부터 떨어진 거리 계산 -> 'distance' 컬럼
pca_clusters = [pca_cluster_0, pca_cluster_1, pca_cluster_2, pca_cluster_3]

for i, pca_cluster in enumerate(pca_clusters):
    tmp = pca_cluster.drop('cluster', axis=1)
    distances = [calculate_distance(centroids[i], point) for point in tmp.values]
    pca_cluster.loc[:,'distance'] = distances

클러스터별 최단 거리 5위 뽑아보기

In [127]:
pd.concat([
    pca_cluster_0.sort_values('distance').head(),
    pca_cluster_1.sort_values('distance').head(),
    pca_cluster_2.sort_values('distance').head(),
    pca_cluster_3.sort_values('distance').head(),
])

,0,1,2,3,4,5,cluster,distance
행정동명,,,,,,,,
송파2동,0.079815,-0.244630,0.027826,0.015311,-0.039013,-0.010445,0,0.143294
남가좌1동,0.088593,-0.312289,0.017162,-0.167120,0.047328,0.041055,0,0.158830
하계2동,0.164587,-0.306063,-0.004596,-0.099306,-0.003715,-0.035986,0,0.163167
방이1동,-0.056980,-0.250338,0.119408,-0.080721,-0.087399,0.055004,0,0.168104
방배3동,0.100324,-0.300655,0.057266,0.061770,0.054492,0.053401,0,0.173730
면목4동,-0.075829,-0.078424,-0.163650,0.052472,0.042259,0.000171,1,0.089658
암사1동,-0.065351,-0.064685,-0.098996,-0.018784,0.066846,0.051345,1,0.093978
홍제3동,-0.044947,-0.102270,-0.078139,0.104911,0.052648,-0.021009,1,0.110024
화곡2동,-0.077616,-0.105319,-0.091817,-0.046530,-0.033173,0.018272,1,0.112856


### 2. stanine을 이용해서 클러스터별 등급 매기기

In [134]:
import pandas as pd
import numpy as np

pca_result = pd.read_csv('./data/minmax_scaling_PCA_label.csv', index_col=0)
original_df = pd.read_csv('./data/final_data_after_winsorization_with_selected_features.csv', index_col=0)

In [ ]:
cluster_0 = original_df.loc[pca_result[pca_result['cluster'] == 0].index, :]
cluster_1 = original_df.loc[pca_result[pca_result['cluster'] == 1].index, :]
cluster_2 = original_df.loc[pca_result[pca_result['cluster'] == 2].index, :]
cluster_3 = original_df.loc[pca_result[pca_result['cluster'] == 3].index, :]

In [ ]:
from sklearn.preprocessing import *

- 클러스터 0

In [ ]:
scaled_data0 = StandardScaler().fit_transform(cluster_0)
data_scale0=pd.DataFrame(data=scaled_data0, columns=cluster_0.columns, index=cluster_0.index)

In [ ]:
def stanine0(z):
    s = (2.5 * z + 5.4).astype(int)
    s[s < 1] = 1
    s[s > 9] = 9
    return s

def stanine0_1(z):
    s = (2 * z + 5.5).astype(int)
    s[s < 1] = 1
    s[s > 9] = 9
    return s


stanine0=stanine0(data_scale0)
stanine0['제로웨이스트 가게수']=stanine0_1(data_scale0['제로웨이스트 가게수'])
stanine0['도시농업(프로그램수)']=stanine0_1(data_scale0['도시농업(프로그램수)'])

In [ ]:
stanine0['등급합']=stanine0['미세먼지지수']+ stanine0['공원수']+stanine0['도시농업(프로그램수)']+stanine0['거주_유동_인구당_매출액']+stanine0['채소가게수']+stanine0['제로웨이스트 가게수']+stanine0['승하차인원_버스_지하철']+stanine0['카페수']+stanine0['유치원_초등학교']+stanine0['인구수']+stanine0['총_10대']+stanine0['총_20_30대']+stanine0['총_60대이상']+stanine0['1인세대']

In [ ]:
stanine0.sort_values('등급합',ascending=False).head(10)

,미세먼지지수,공원수,도시농업(프로그램수),거주_유동_인구당_매출액,채소가게수,제로웨이스트 가게수,승하차인원_버스_지하철,카페수,유치원_초등학교,인구수,총_10대,총_20_30대,총_60대이상,1인세대,등급합
행정동명,,,,,,,,,,,,,,,
오류2동,7,9,4,3,9,7,5,5,9,5,4,5,9,7,88
역삼2동,4,9,4,3,9,7,8,9,2,5,8,6,1,9,84
청운효자동,9,3,4,6,5,7,5,7,7,3,3,7,7,9,82
청담동,4,6,4,4,5,4,9,8,8,3,3,8,4,9,79
삼성1동,4,4,4,6,5,4,9,9,7,1,3,7,6,9,78
강일동,6,8,9,3,6,4,5,3,9,4,4,5,7,5,78
삼성2동,4,2,7,7,8,4,8,9,1,4,5,8,2,9,78
방배4동,4,3,4,5,8,7,6,6,3,7,5,7,4,8,77
신도림동,4,4,4,4,4,7,9,7,6,5,4,8,6,5,77


In [ ]:
stanine0.describe().round()

,미세먼지지수,공원수,도시농업(프로그램수),거주_유동_인구당_매출액,채소가게수,제로웨이스트 가게수,승하차인원_버스_지하철,카페수,유치원_초등학교,인구수,총_10대,총_20_30대,총_60대이상,1인세대,등급합
count,76.0,76.0,76.0,76.0,76.0,76.0,76.0,76.0,76.0,76.0,76.0,76.0,76.0,76.0,76.0
mean,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,67.0
std,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,8.0
min,4.0,2.0,4.0,3.0,2.0,4.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,53.0
25%,4.0,3.0,4.0,3.0,4.0,4.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,62.0
50%,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,5.0,5.0,4.0,5.0,5.0,5.0,67.0
75%,4.0,6.0,4.0,6.0,6.0,7.0,5.0,6.0,7.0,6.0,7.0,7.0,6.0,7.0,71.0
max,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,88.0


- 클러스터 1

In [ ]:
scaled_data1 = StandardScaler().fit_transform(cluster_1)
data_scale1=pd.DataFrame(data=scaled_data1, columns=cluster_1.columns, index=cluster_1.index)

In [ ]:
def stanine1(z):
    s = (2.5 * z + 5.5).astype(int)
    s[s < 1] = 1
    s[s > 9] = 9
    return s

def stanine1_0(z):
    s = (2 * z + 6).astype(int)
    s[s < 1] = 1
    s[s > 9] = 9
    return s

stanine1=stanine1(data_scale1)
stanine1['도시농업(프로그램수)']=stanine1_0(data_scale1['도시농업(프로그램수)'])

In [ ]:
stanine1['등급합']=stanine1['미세먼지지수']+ stanine1['공원수']+stanine1['도시농업(프로그램수)']+stanine1['거주_유동_인구당_매출액']+stanine1['채소가게수']+stanine1['제로웨이스트 가게수']+stanine1['승하차인원_버스_지하철']+stanine1['카페수']+stanine1['유치원_초등학교']+stanine1['인구수']+stanine1['총_10대']+stanine1['총_20_30대']+stanine1['총_60대이상']+stanine1['1인세대']

In [ ]:
stanine1.sort_values('등급합').head(10)

,미세먼지지수,공원수,도시농업(프로그램수),거주_유동_인구당_매출액,채소가게수,제로웨이스트 가게수,승하차인원_버스_지하철,카페수,유치원_초등학교,인구수,총_10대,총_20_30대,총_60대이상,1인세대,등급합
행정동명,,,,,,,,,,,,,,,
면목5동,4,5,5,5,4,4,3,3,1,4,4,5,3,2,52
반포본동,5,6,5,4,4,4,2,1,9,1,1,1,1,8,52
이촌2동,5,3,5,4,4,9,2,2,1,1,4,4,6,3,53
창신2동,4,4,5,3,5,4,3,3,1,1,2,3,8,7,53
동화동,4,4,5,6,4,4,1,4,3,2,8,3,5,1,54
효창동,5,4,5,5,4,4,3,2,3,2,5,8,1,3,54
난향동,4,3,5,6,4,4,3,2,4,4,9,1,4,1,54
금호1가동,4,3,5,4,4,4,3,3,4,4,6,4,4,2,54
청림동,5,1,5,7,4,4,2,2,5,4,7,4,4,1,55


In [ ]:
stanine1.describe().round()

,미세먼지지수,공원수,도시농업(프로그램수),거주_유동_인구당_매출액,채소가게수,제로웨이스트 가게수,승하차인원_버스_지하철,카페수,유치원_초등학교,인구수,총_10대,총_20_30대,총_60대이상,1인세대,등급합
count,177.0,177.0,177.0,177.0,177.0,177.0,177.0,177.0,177.0,177.0,177.0,177.0,177.0,177.0,177.0
mean,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,69.0
std,1.0,2.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,8.0
min,4.0,1.0,5.0,3.0,4.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,52.0
25%,4.0,3.0,5.0,4.0,4.0,4.0,3.0,3.0,3.0,3.0,4.0,3.0,4.0,3.0,64.0
50%,4.0,5.0,5.0,4.0,4.0,4.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,69.0
75%,5.0,6.0,5.0,5.0,5.0,4.0,6.0,6.0,6.0,7.0,7.0,7.0,6.0,7.0,75.0
max,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,92.0


- 클러스터 2

In [ ]:
scaled_data2 = StandardScaler().fit_transform(cluster_2)
data_scale2=pd.DataFrame(data=scaled_data2, columns=cluster_2.columns, index=cluster_2.index)

In [ ]:
def stanine2(z):
    s = (2.5 * z + 5.5).astype(int)
    s[s < 1] = 1
    s[s > 9] = 9
    return s

def stanine2_1(z):
    s = (4 * z + 6.3).astype(int)
    s[s < 1] = 1
    s[s > 9] = 9
    return s

stanine2=stanine2(data_scale2)
stanine2['미세먼지지수']=stanine2_1(data_scale2['미세먼지지수'])

In [ ]:
def stanine2(z):
    s = (2.5 * z + 5.5).astype(int)
    s[s < 1] = 1
    s[s > 9] = 9
    return s

def stanine2_1(z):
    s = (4 * z + 6.3).astype(int)
    s[s < 1] = 1
    s[s > 9] = 9
    return s

stanine2=stanine2(data_scale2)
stanine2['미세먼지지수']=stanine2_1(data_scale2['미세먼지지수'])

In [ ]:
stanine2['등급합']=stanine2['미세먼지지수']+ stanine2['공원수']+stanine2['도시농업(프로그램수)']+stanine2['거주_유동_인구당_매출액']+stanine2['채소가게수']+stanine2['제로웨이스트 가게수']+stanine2['승하차인원_버스_지하철']+stanine2['카페수']+stanine2['유치원_초등학교']+stanine2['인구수']+stanine2['총_10대']+stanine2['총_20_30대']+stanine2['총_60대이상']+stanine2['1인세대']

In [ ]:
stanine2.sort_values('등급합', ascending=False).head(10)

,미세먼지지수,공원수,도시농업(프로그램수),거주_유동_인구당_매출액,채소가게수,제로웨이스트 가게수,승하차인원_버스_지하철,카페수,유치원_초등학교,인구수,총_10대,총_20_30대,총_60대이상,1인세대,등급합
행정동명,,,,,,,,,,,,,,,
진관동,5,9,4,4,4,7,8,8,9,9,9,6,8,3,93
상암동,4,9,9,9,3,9,9,9,8,4,9,3,2,4,91
화곡1동,4,8,4,7,4,9,7,6,4,9,2,9,9,8,90
양재1동,8,9,4,4,4,3,9,8,9,9,7,4,5,4,87
성산2동,7,7,7,4,9,7,4,4,5,8,5,7,3,7,84
상도1동,9,3,4,4,4,5,9,5,5,9,5,9,5,8,84
가양1동,8,8,4,7,4,9,5,9,3,6,2,9,1,9,84
연희동,4,6,7,5,4,9,9,8,2,6,3,8,4,9,84
독산1동,4,5,9,4,4,5,8,5,6,9,3,9,4,8,83


In [ ]:
stanine2.describe().round()

,미세먼지지수,공원수,도시농업(프로그램수),거주_유동_인구당_매출액,채소가게수,제로웨이스트 가게수,승하차인원_버스_지하철,카페수,유치원_초등학교,인구수,총_10대,총_20_30대,총_60대이상,1인세대,등급합
count,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
mean,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,68.0
std,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,10.0
min,3.0,1.0,4.0,2.0,2.0,3.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,48.0
25%,4.0,3.0,4.0,3.0,4.0,3.0,3.0,4.0,3.0,3.0,3.0,4.0,3.0,4.0,60.0
50%,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,5.0,5.0,4.0,4.0,5.0,5.0,68.0
75%,5.0,6.0,4.0,6.0,6.0,5.0,6.0,6.0,6.0,7.0,6.0,6.0,7.0,7.0,75.0
max,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,93.0


- 클러스터 3

In [ ]:
scaled_data3 = StandardScaler().fit_transform(cluster_3)
data_scale3=pd.DataFrame(data=scaled_data3, columns=cluster_3.columns, index=cluster_3.index)

In [ ]:
def stanine3(z):
    s = (2.6 * z + 5.7).astype(int)
    s[s < 1] = 1
    s[s > 9] = 9
    return s

def stanine3_1(z):
    s = (3 * z + 6).astype(int)
    s[s < 1] = 1
    s[s > 9] = 9
    return s

def stanine3_2(z):
    s = (4 * z + 6).astype(int)
    s[s < 1] = 1
    s[s > 9] = 9
    return s

def stanine3_3(z):
    s = (2 * z + 5.7).astype(int)
    s[s < 1] = 1
    s[s > 9] = 9
    return s

stanine3=stanine3(data_scale1)
stanine3['미세먼지지수']=stanine3_1(data_scale1['미세먼지지수'])
stanine3['채소가게수']=stanine3_2(data_scale1['채소가게수'])
stanine3['도시농업(프로그램수)']=stanine3_3(data_scale1['도시농업(프로그램수)'])

In [ ]:
stanine3['등급합']=stanine3['미세먼지지수']+ stanine3['공원수']+stanine3['도시농업(프로그램수)']+stanine3['거주_유동_인구당_매출액']+stanine3['채소가게수']+stanine3['제로웨이스트 가게수']+stanine3['승하차인원_버스_지하철']+stanine3['카페수']+stanine3['유치원_초등학교']+stanine3['인구수']+stanine3['총_10대']+stanine3['총_20_30대']+stanine3['총_60대이상']+stanine3['1인세대']

In [ ]:
stanine3.sort_values('등급합', ascending=False).head(10)

,미세먼지지수,공원수,도시농업(프로그램수),거주_유동_인구당_매출액,채소가게수,제로웨이스트 가게수,승하차인원_버스_지하철,카페수,유치원_초등학교,인구수,총_10대,총_20_30대,총_60대이상,1인세대,등급합
행정동명,,,,,,,,,,,,,,,
길동,5,8,5,8,9,4,7,9,9,8,4,7,5,6,94
자양1동,4,3,9,4,9,9,6,6,5,8,5,9,3,9,89
성북동,9,3,5,4,6,8,9,9,7,5,6,7,4,7,89
제기동,4,4,5,4,9,4,9,8,6,9,3,7,7,9,88
독산3동,4,9,5,4,9,4,9,6,5,9,2,7,6,8,87
월계1동,4,9,5,6,4,8,9,7,4,7,7,6,5,6,87
상계2동,4,1,5,9,7,8,9,9,6,6,9,4,5,4,86
응암3동,4,6,5,5,9,4,6,7,7,8,3,5,8,8,85
방배2동,5,9,5,7,5,9,8,8,2,6,9,5,4,3,85


In [ ]:
stanine3.describe().round()

,미세먼지지수,공원수,도시농업(프로그램수),거주_유동_인구당_매출액,채소가게수,제로웨이스트 가게수,승하차인원_버스_지하철,카페수,유치원_초등학교,인구수,총_10대,총_20_30대,총_60대이상,1인세대,등급합
count,177.0,177.0,177.0,177.0,177.0,177.0,177.0,177.0,177.0,177.0,177.0,177.0,177.0,177.0,177.0
mean,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,71.0
std,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,8.0
min,4.0,1.0,5.0,3.0,3.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,52.0
25%,4.0,3.0,5.0,4.0,4.0,4.0,3.0,3.0,3.0,3.0,4.0,4.0,4.0,3.0,66.0
50%,4.0,5.0,5.0,4.0,5.0,4.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,71.0
75%,5.0,6.0,5.0,5.0,6.0,4.0,6.0,7.0,6.0,7.0,7.0,7.0,6.0,7.0,77.0
max,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,94.0


## 최종 입지 선정